In [1]:
from ctypes import *
import os
from matplotlib import pyplot as py
import numpy as np
from inspect import getfullargspec
import glob
from mnist import MNIST
import random
import idx2numpy
%matplotlib inline

In [1]:
def imList(im): 
    if not isinstance(im, list):
        X=[] #Create an empty list to store the numpy array (image)
        if(len(np.shape(im)) > 2):
            for ima in im:
                X.append(ima)
        else:
            X.append(im) #Store the image in a list (For future preparation when we need to import multiple images)
    else: 
        X=im
    return X

#### Plan: Focus on a single image. When you can make it work for a single image, branch off and consolidate from there. Current method only imports a single image.

In [5]:
def flifPress(im): 
    LIB = CDLL ("./lib/libflif.dll") #import libflif library
    X=imList(im) #Will fix this function up later
    
    encfunc=LIB.flif_create_encoder #Set up function 
    encfunc.restype=POINTER(c_float) #declare a pointer to the function
    enc=encfunc() #declare the function. Same thing will be used later. 
    fim=[]
    [FW2, FH2] = max(np.shape(matx) for matx in X)

    for i in range(len(X)):
        
        ###Next thing to work on is to fix the padding to pad the least values###
        [curW, curH] = np.shape(X[i]);
        padW = FW2 - curW +1; padH = FH2 - curH+1;
        # if difference between length and height are greater than 1, pad the difference
    
        X[i] = np.pad(X[i],((0,padW),(0,padH))) 
        [W, H] = np.shape(X[i]) #New Width x Height
        pim = X[i].ctypes.data_as(POINTER(c_float)) #calling the pointer at the numpy array.
              
        fimfunc=LIB.flif_import_image_GRAY #Set up function
        fimfunc.restype=POINTER(c_void_p) #Set up pointer to the function
        
        temp=fimfunc(W, H, pim, W) #set Fim up to use the function, now that it's establisehd with a pointer
        fim.append(temp)
        LIB.flif_encoder_add_image_move(enc, fim[i]) #Use function to return a pointer
            
    #####For Output##########
    pdest=pointer(c_uint8()) #Setting up Double Pointer. Will need to pass this on as a byref for double pointer
    prsz=c_size_t(0);
    
    encmem=LIB.flif_encoder_encode_memory
    encmem(enc, byref(pdest), byref(prsz))
    nBytes=prsz.value
    
    #########Clean Up#######################
    for fimi in fim: 
        LIB.flif_destroy_image(fimi)
    LIB.flif_destroy_encoder(enc)
    LIB.flif_free_memory(pdest)
    return nBytes

In [4]:
#n=flifPress(image_list)
#print(n)

In [5]:
#Construct Test Case####
#cOMMAND LINe flif
####NIST digits
#Convert Training bUsedTraining

## Testing Code Here
#### Running test values. 
#### ztest zero array
#### BWim - BW .png file
#### BWtest -result

In [6]:
ztest = np.zeros((512,512), dtype='uint8')

In [7]:
test=flifPress(ztest)
print(test)

16


In [4]:
BWim=py.imread('lena.png')
print(type(BWim[0][0]))

NameError: name 'py' is not defined

In [3]:
BWtest=flifPress(BWim)
print(BWtest)

NameError: name 'BWim' is not defined